In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
from tqdm.notebook import tqdm
import logging
import time

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from bisect import bisect_left
from scipy.stats import rankdata
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import optuna
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

In [27]:
RANDOM_STATE = 5
REPEATS = 3
sk = StratifiedKFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)

In [28]:
train = pd.read_csv(r'train.csv (1)\train.csv')
# test = pd.read_csv('../input/playground-series-s3e24/test.csv')

In [29]:
X = train.drop(['id', 'smoking'], axis=1)
y = train['smoking']

# X_test = test.drop('id', axis=1)

# Preprocessing

In [30]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   age                  159256 non-null  int64  
 1   height(cm)           159256 non-null  int64  
 2   weight(kg)           159256 non-null  int64  
 3   waist(cm)            159256 non-null  float64
 4   eyesight(left)       159256 non-null  float64
 5   eyesight(right)      159256 non-null  float64
 6   hearing(left)        159256 non-null  int64  
 7   hearing(right)       159256 non-null  int64  
 8   systolic             159256 non-null  int64  
 9   relaxation           159256 non-null  int64  
 10  fasting blood sugar  159256 non-null  int64  
 11  Cholesterol          159256 non-null  int64  
 12  triglyceride         159256 non-null  int64  
 13  HDL                  159256 non-null  int64  
 14  LDL                  159256 non-null  int64  
 15  hemoglobin       

In [31]:
intial_cat_features = ['dental caries', 'hearing(left)', 'hearing(right)', 'Urine protein']
num_features = [col for col in X.columns if col not in intial_cat_features]

In [32]:
# for feat in intial_cat_features:
#     X[feat] = X[feat].astype(str)

In [33]:
# X.info()

In [34]:
X.shape

(159256, 22)

In [35]:
standard_scaler = StandardScaler().fit_transform(X[num_features])
# robust_scaler = RobustScaler().fit(X[num_features])
train_scaled = pd.DataFrame(standard_scaler, columns=num_features)
X = pd.concat([X[intial_cat_features], train_scaled], axis=1)
X.head()

,dental caries,hearing(left),hearing(right),Urine protein,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,serum creatinine,AST,ALT,Gtp
0,0,1,1,1,0.902985,-0.030268,-0.567581,-0.223489,-1.257856,-1.022156,0.981702,1.125777,-0.283930,-0.837985,2.604429,-1.135246,-1.406581,1.189928,0.597927,-0.371570,-0.087326,-0.295342
1,1,2,2,1,2.169636,-0.030268,-0.170319,0.669577,-1.009169,-0.767247,1.845852,0.681066,3.173413,-0.063252,-1.097105,0.082162,0.404573,0.980315,1.155511,0.156700,-0.199983,0.025124
2,0,1,1,1,-2.052535,0.536694,0.624205,-0.223489,-1.506543,-1.277064,-0.353802,-0.208355,-1.262423,-0.626695,1.048274,-0.777185,-0.767350,1.818767,-0.517239,0.156700,0.250645,0.537870
3,1,1,1,1,-0.785883,1.670617,2.213252,2.455708,1.229017,0.507296,0.667465,1.236955,-0.479628,-0.556264,1.138923,-1.278470,-0.447734,0.770702,0.597927,-0.582878,0.025331,-0.199202
4,0,1,1,1,-1.208100,-0.030268,-0.567581,-0.279305,1.229017,-0.002521,-0.118125,-0.097177,-0.479628,-1.436643,-0.613639,-0.848797,-0.767350,0.421347,-0.517239,-0.688532,-0.763267,-0.615808


# Cross-Validation

In [36]:
%%time

# Initialize lists for scores
hist_scores, lgbm_scores, xgb_scores, cat_scores = [], [], [], []
ada_scores, grad_scores, logreg_scores, ridge_scores = [], [], [], []
gnb_scores, lda_scores = [], []
     
for i, (train_idx, test_idx) in enumerate(sk.split(X, y)):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    print('----------------------------------------------------------')

    ##########################
    ## HistGradientBoosting ##
    ##########################

    start_time = time.time()
    hist = HistGradientBoostingClassifier(random_state=5).fit(X_train, y_train)
    
    hist_train_pred = hist.predict_proba(X_train)[:, 1]
    hist_pred = hist.predict_proba(X_test)[:, 1]
    hist_score = roc_auc_score(y_test, hist_pred)
    hist_scores.append(hist_score)
    
    print(f'Fold {i + 1} ==> HistGradientBoosting ROC-AUC score is ==> {hist_score}')
    print(f'{time.time() - start_time}')
          
    ##########
    ## LGBM ##
    ##########

    start_time = time.time()
    lgbm = LGBMClassifier(n_jobs=-1, random_state=5).fit(X_train, y_train)

    lgbm_train_pred = lgbm.predict_proba(X_train)[:, 1]
    lgbm_pred = lgbm.predict_proba(X_test)[:, 1]
    lgbm_score = roc_auc_score(y_test, lgbm_pred)
    lgbm_scores.append(lgbm_score)
    
    print(f'Fold {i + 1} ==> LGBM ROC-AUC score is ==> {lgbm_score}')
    print(f'{time.time() - start_time}')

    #############
    ## XGBoost ##
    #############

    start_time = time.time()
    xgb = XGBClassifier(seed=5).fit(X_train, y_train)

    xgb_train_pred = xgb.predict_proba(X_train)[:, 1]
    xgb_pred = xgb.predict_proba(X_test)[:, 1]
    xgb_score = roc_auc_score(y_test, xgb_pred)
    xgb_scores.append(xgb_score)
    
    print(f'Fold {i + 1} ==> XGBoost ROC-AUC score is ==> {xgb_score}')
    print(f'{time.time() - start_time}')

    ##############
    ## CatBoost ##
    ##############

    start_time = time.time()
    cat = CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100, cat_features=intial_cat_features).fit(X_train, y_train)

    cat_train_pred = cat.predict_proba(X_train)[:, 1]
    cat_pred = cat.predict_proba(X_test)[:, 1]
    cat_score = roc_auc_score(y_test, cat_pred)
    cat_scores.append(cat_score)
    
    print(f'Fold {i + 1} ==> CatBoost ROC-AUC score is ==> {cat_score}')
    print(f'{time.time() - start_time}')

    ########################
    ## AdaBoostClassifier ##
    ########################
    
    start_time = time.time()
    ada = AdaBoostClassifier(random_state=5).fit(X_train, y_train)
    ada_pred = ada.predict_proba(X_test)[:, 1]
    ada_score = roc_auc_score(y_test, ada_pred)
    ada_scores.append(ada_score)

    print(f'Fold {i + 1} ==> AdaBoostClassifier ROC-AUC score is ==> {ada_score}')
    print(f'{time.time() - start_time}')
    
    ################################
    ## GradientBoostingClassifier ##
    ################################
    
    start_time = time.time()
    grad = GradientBoostingClassifier(random_state=5).fit(X_train, y_train)
    grad_pred = grad.predict_proba(X_test)[:, 1]
    grad_score = roc_auc_score(y_test, grad_pred)
    grad_scores.append(grad_score)

    print(f'Fold {i + 1} ==> GradientBoostingClassifier ROC-AUC score is ==> {grad_score}')
    print(f'{time.time() - start_time}')
    
    ########################
    ## LogisticRegression ##
    ########################
    
    start_time = time.time()
    logreg = LogisticRegression(random_state=5).fit(X_train, y_train)
    logreg_pred = logreg.predict_proba(X_test)[:, 1]
    logreg_score = roc_auc_score(y_test, logreg_pred)
    logreg_scores.append(logreg_score)

    print(f'Fold {i + 1} ==> LogisticRegression ROC-AUC score is ==> {logreg_score}')
    print(f'{time.time() - start_time}')
    
    #####################
    ## RidgeClassifier ##
    #####################
    
    start_time = time.time()
    ridge = RidgeClassifier(random_state=5).fit(X_train, y_train)
    ridge_pred = ridge.decision_function(X_test)
    ridge_score = roc_auc_score(y_test, ridge_pred)
    ridge_scores.append(ridge_score)

    print(f'Fold {i + 1} ==> RidgeClassifier ROC-AUC score is ==> {ridge_score}')
    print(f'{time.time() - start_time}')
    
    ################
    ## GaussianNB ##
    ################
    
    start_time = time.time()
    gnb = GaussianNB().fit(X_train, y_train)
    gnb_pred = gnb.predict_proba(X_test)[:, 1]
    gnb_score = roc_auc_score(y_test, gnb_pred)
    gnb_scores.append(gnb_score)

    print(f'Fold {i + 1} ==> GaussianNB ROC-AUC score is ==> {gnb_score}')
    print(f'{time.time() - start_time}')
    
    ################################
    ## LinearDiscriminantAnalysis ##
    ################################
    
    start_time = time.time()
    lda = LinearDiscriminantAnalysis().fit(X_train, y_train)
    lda_pred = lda.predict_proba(X_test)[:, 1]
    lda_score = roc_auc_score(y_test, lda_pred)
    lda_scores.append(lda_score)

    print(f'Fold {i + 1} ==> LinearDiscriminantAnalysis ROC-AUC score is ==> {lda_score}')
    print(f'{time.time() - start_time}')

    print()
    
print('----------------------------------------------------------')
print(f'Average Hist Score is ==> {np.mean(hist_scores)}')
print(f'Average LGBM Score is ==> {np.mean(lgbm_scores)}')
print(f'Average XGB Score is ==> {np.mean(xgb_scores)}')
print(f'Average CAT Score is ==> {np.mean(cat_scores)}')
print(f'Average AdaBoost Score: {np.mean(ada_scores)}')
print(f'Average GradientBoosting Score: {np.mean(grad_scores)}')
print(f'Average Logistic Regression Score: {np.mean(logreg_scores)}')
print(f'Average Ridge Classifier Score: {np.mean(ridge_scores)}')
print(f'Average GaussianNB Score: {np.mean(gnb_scores)}')
print(f'Average LDA Score: {np.mean(lda_scores)}')

----------------------------------------------------------
Fold 1 ==> HistGradientBoosting ROC-AUC score is ==> 0.8604118601574998
2.5751607418060303
Fold 1 ==> LGBM ROC-AUC score is ==> 0.86044976749313
1.4477119445800781
Fold 1 ==> XGBoost ROC-AUC score is ==> 0.8620293386115212
5.524072170257568
Fold 1 ==> CatBoost ROC-AUC score is ==> 0.8646036648671098
129.14887976646423
Fold 1 ==> AdaBoostClassifier ROC-AUC score is ==> 0.8463182059672492
9.650861263275146
Fold 1 ==> GradientBoostingClassifier ROC-AUC score is ==> 0.8540488671378328
35.06326675415039
Fold 1 ==> LogisticRegression ROC-AUC score is ==> 0.8278362409509454
0.6018974781036377
Fold 1 ==> RidgeClassifier ROC-AUC score is ==> 0.8254450346837702
0.1186072826385498
Fold 1 ==> GaussianNB ROC-AUC score is ==> 0.7862009287457447
0.16360759735107422
Fold 1 ==> LinearDiscriminantAnalysis ROC-AUC score is ==> 0.8254451468356507
0.5556588172912598

----------------------------------------------------------
Fold 2 ==> HistGradient